# Ada Carbon Monitoring System - API Data Provider Components

This notebook documents the carbon monitoring components that serve data to the Ada UI widgets through the Ada API.

## System Architecture

```
┌─────────────┐      ┌─────────────┐      ┌─────────────────────┐
│   Ada UI    │─────>│   Ada API   │─────>│  Carbon Monitoring  │
│  (Svelte)   │<─────│  (FastAPI)  │<─────│    Components       │
└─────────────┘      └─────────────┘      └─────────────────────┘
                                                     │
                          ┌──────────────────────────┼──────────────────────────┐
                          │                          │                          │
                          ▼                          ▼                          ▼
                   ┌─────────────┐          ┌─────────────┐          ┌─────────────┐
                   │  MongoDB    │          │ Prometheus  │          │   Carbon    │
                   │   Client    │          │ API Client  │          │  Intensity  │
                   │             │          │             │          │ API Client  │
                   └─────────────┘          └─────────────┘          └─────────────┘
```

## Widgets to Serve

1. **Stacked Bar Chart** - Busy/Idle usage over time (electricity or carbon)
2. **GitHub-style Heatmap** - Daily carbon usage for full year
3. **Workspace Tracker Card** - Detailed workspace metrics

## Table of Contents
1. [Component Specifications](#specifications)
2. [Data Aggregator Components](#aggregators)
3. [Calculation Components](#calculators)
4. [Data Model Components](#models)
5. [Widget Data Flow Examples](#widgets)
6. [API Response Examples](#api-responses)

In [ ]:
# Setup
import sys
import os
from datetime import datetime, timedelta, timezone
import json

# Ensure the source directory is in the path
sys.path.append(os.path.dirname(os.path.abspath('__file__')))

## 1. Component Specifications {#specifications}

Each component is a data provider with defined inputs and outputs.

### Component Categories

**Data Aggregators** - Fetch raw data from external sources
- MongoDBClient
- PrometheusAPIClient
- CarbonIntensityAPIClient

**Calculators** - Transform raw data into metrics
- ElectricityEstimator
- CarbonCalculator
- CarbonEquivalencyCalculator

**Data Models** - Structure and validate complete records
- WorkspaceUsageEntry

## 2. Data Aggregator Components {#aggregators}

### 2.1 MongoDBClient - User Attribution Provider

In [ ]:
print("MongoDBClient - Input/Output Specification")
print("="*60)

spec = {
    "component": "MongoDBClient",
    "purpose": "Attribute workspace usage to users/groups via time-based lookup",
    "methods": [
        {
            "name": "get_user_by_host_and_time",
            "inputs": {
                "hostname": "str - IP address (e.g., '172.16.100.50')",
                "timestamp": "datetime - When to check workspace ownership"
            },
            "outputs": {
                "success": {
                    "platform_name": "str - User identifier",
                    "workspace_id": "ObjectId - MongoDB workspace ID"
                },
                "failure": "None - No workspace active at that time"
            },
            "use_case": "Find which user was using a host at specific timestamp"
        },
        {
            "name": "get_user_by_platform_name",
            "inputs": {
                "platform_name": "str - User identifier from workspace"
            },
            "outputs": {
                "success": {
                    "platform_name": "str",
                    "name": "str - Full name",
                    "email": "str - Email address",
                    "uid": "int - Unix user ID"
                },
                "failure": "None - User not found"
            },
            "use_case": "Enrich usage records with complete user details"
        },
        {
            "name": "get_workspaces_by_hostname",
            "inputs": {
                "hostname": "str - IP address",
                "start_time": "datetime - Range start (optional)",
                "end_time": "datetime - Range end (optional)"
            },
            "outputs": {
                "success": "List[Dict] - All workspaces in time range",
                "failure": "[] - Empty list if none found"
            },
            "use_case": "Batch process all workspaces for a host"
        },
        {
            "name": "attribute_usage_to_user",
            "inputs": {
                "workspace_id": "ObjectId - Workspace identifier",
                "usage_entry": "Dict - Complete usage record"
            },
            "outputs": {
                "success": "InsertOneResult - MongoDB insert result",
                "failure": "Exception - Database error"
            },
            "use_case": "Store usage record linked to workspace"
        }
    ]
}

print(json.dumps(spec, indent=2))

print("\n" + "="*60)
print("Example: Time-based User Lookup")
print("="*60)

example_input = {
    "hostname": "172.16.100.50",
    "timestamp": "2025-01-15T12:00:00Z"
}

example_output = {
    "platform_name": "john.doe",
    "workspace_id": "507f1f77bcf86cd799439011"
}

print(f"Input: {json.dumps(example_input, indent=2)}")
print(f"\nOutput: {json.dumps(example_output, indent=2)}")

### 2.2 PrometheusAPIClient - CPU Metrics Provider

In [ ]:
print("PrometheusAPIClient - Input/Output Specification")
print("="*60)

spec = {
    "component": "PrometheusAPIClient",
    "purpose": "Fetch CPU usage metrics from Prometheus time-series database",
    "methods": [
        {
            "name": "query",
            "inputs": {
                "parameters": {
                    "query": "str - PromQL query string",
                    "start": "str - RFC3339 timestamp",
                    "end": "str - RFC3339 timestamp",
                    "step": "str - Time resolution (e.g., '1h')"
                }
            },
            "outputs": {
                "success": {
                    "status": "'success'",
                    "data": {
                        "result": "List[Dict] - Metric time series"
                    }
                },
                "failure": "None - On timeout or error"
            },
            "use_case": "Generic Prometheus query execution"
        },
        {
            "name": "cpu_seconds_total",
            "inputs": {
                "timestamp": "datetime - Query timestamp",
                "cloud_project_name": "str - Project filter (required)",
                "machine_name": "str - Machine filter (optional)",
                "host": "str - Host IP filter (optional)",
                "step": "str - Time window (default: '1h')"
            },
            "outputs": {
                "success": {
                    "status": "'success'",
                    "data": {
                        "result": [
                            {
                                "metric": {"mode": "user|idle"},
                                "values": [["timestamp", "cpu_seconds"]]
                            }
                        ]
                    }
                },
                "failure": "None - On timeout or error"
            },
            "use_case": "Get CPU time increase for specific host/project"
        },
        {
            "name": "_to_rfc3339",
            "inputs": {
                "date": "datetime - Timestamp to format"
            },
            "outputs": {
                "success": "str - RFC3339 formatted string (e.g., '2025-01-15T12:00:00Z')"
            },
            "use_case": "Convert Python datetime to Prometheus-compatible format"
        }
    ]
}

print(json.dumps(spec, indent=2))

print("\n" + "="*60)
print("Example: CPU Metrics Query")
print("="*60)

example_input = {
    "timestamp": "2025-01-15T12:00:00Z",
    "cloud_project_name": "CDAaaS",
    "machine_name": "MUON",
    "host": "172.16.100.50",
    "step": "1h"
}

example_output = {
    "status": "success",
    "data": {
        "result": [
            {
                "metric": {"mode": "user", "cpu": "0"},
                "values": [[1736942400, "2400.0"]]
            },
            {
                "metric": {"mode": "idle", "cpu": "0"},
                "values": [[1736942400, "1200.0"]]
            }
        ]
    }
}

print("Input:")
print(json.dumps(example_input, indent=2))
print("\nOutput:")
print(json.dumps(example_output, indent=2))
print("\nParsed:")
print(f"  Busy CPU: {example_output['data']['result'][0]['values'][0][1]} seconds")
print(f"  Idle CPU: {example_output['data']['result'][1]['values'][0][1]} seconds")

### 2.3 CarbonIntensityAPIClient - Grid Carbon Data Provider

In [ ]:
print("CarbonIntensityAPIClient - Input/Output Specification")
print("="*60)

spec = {
    "component": "CarbonIntensityAPIClient",
    "purpose": "Fetch UK grid carbon intensity from Carbon Intensity API",
    "methods": [
        {
            "name": "get_carbon_intensity",
            "inputs": {
                "timestamp": "datetime - When to get carbon intensity"
            },
            "outputs": {
                "success": "float - Carbon intensity in gCO2/kWh (averaged)",
                "failure": "None - On API error or missing data"
            },
            "behavior": [
                "1. Queries ±15 min window around timestamp",
                "2. Falls back to wider window if needed",
                "3. Averages 'actual' values from multiple periods",
                "4. Returns single float for simplicity"
            ],
            "use_case": "Get grid carbon intensity for energy-to-carbon conversion"
        }
    ]
}

print(json.dumps(spec, indent=2))

print("\n" + "="*60)
print("Example: Carbon Intensity Lookup")
print("="*60)

example_input = {
    "timestamp": "2025-01-15T12:00:00Z"
}

# API returns multiple 30-min periods
api_response = {
    "data": [
        {
            "from": "2025-01-15T11:45:00Z",
            "to": "2025-01-15T12:00:00Z",
            "intensity": {"actual": 47, "forecast": 45}
        },
        {
            "from": "2025-01-15T12:00:00Z",
            "to": "2025-01-15T12:15:00Z",
            "intensity": {"actual": 44, "forecast": 43}
        }
    ]
}

# Client returns averaged value
example_output = 45.5  # (47 + 44) / 2

print("Input:")
print(json.dumps(example_input, indent=2))
print("\nAPI Response (internal):")
print(json.dumps(api_response, indent=2))
print("\nClient Output:")
print(f"  {example_output} gCO2/kWh")
print("\nNote: Client handles API complexities, returns single float")

## 3. Calculation Components {#calculators}

### 3.1 ElectricityEstimator - CPU Time → kWh Converter

In [ ]:
from electricity.ElectricityEstimator import ElectricityEstimator

print("ElectricityEstimator - Input/Output Specification")
print("="*60)

spec = {
    "component": "ElectricityEstimator",
    "purpose": "Convert CPU time to energy consumption (kWh)",
    "constants": {
        "busy_power_w": "12.0 - Watts when CPU is busy",
        "idle_power_w": "1.0 - Watts when CPU is idle"
    },
    "methods": [
        {
            "name": "estimate_usage_kwh",
            "inputs": {
                "busy_cpu_seconds": "float - Seconds of busy CPU time",
                "idle_cpu_seconds": "float - Seconds of idle CPU time"
            },
            "outputs": {
                "busy_kwh": "float - Energy from busy time",
                "idle_kwh": "float - Energy from idle time",
                "total_kwh": "float - Total energy consumption"
            },
            "formula": [
                "busy_kwh = (busy_cpu_seconds * busy_power_w) / 3600000",
                "idle_kwh = (idle_cpu_seconds * idle_power_w) / 3600000",
                "total_kwh = busy_kwh + idle_kwh"
            ],
            "use_case": "Primary method for energy calculation"
        },
        {
            "name": "get_power_consumption_breakdown",
            "inputs": {
                "busy_cpu_seconds": "float",
                "idle_cpu_seconds": "float"
            },
            "outputs": {
                "busy_percentage": "float - % of time busy",
                "idle_percentage": "float - % of time idle",
                "average_power_w": "float - Average power draw"
            },
            "use_case": "Analysis and visualization breakdown"
        }
    ]
}

print(json.dumps(spec, indent=2))

print("\n" + "="*60)
print("Example: CPU Time to Energy Conversion")
print("="*60)

estimator = ElectricityEstimator(busy_power_w=12.0, idle_power_w=1.0)

# Example: 1 hour total (40 min busy, 20 min idle)
example_input = {
    "busy_cpu_seconds": 2400.0,
    "idle_cpu_seconds": 1200.0
}

example_output = estimator.estimate_usage_kwh(
    busy_cpu_seconds=example_input["busy_cpu_seconds"],
    idle_cpu_seconds=example_input["idle_cpu_seconds"]
)

print("Input:")
print(json.dumps(example_input, indent=2))
print(f"  = {example_input['busy_cpu_seconds']/60:.1f} minutes busy")
print(f"  + {example_input['idle_cpu_seconds']/60:.1f} minutes idle")
print("\nOutput:")
print(json.dumps(example_output, indent=2))
print("\nCalculation:")
print(f"  Busy: {example_input['busy_cpu_seconds']}s × 12W / 3600000 = {example_output['busy_kwh']:.6f} kWh")
print(f"  Idle: {example_input['idle_cpu_seconds']}s × 1W / 3600000 = {example_output['idle_kwh']:.6f} kWh")

### 3.2 CarbonCalculator - kWh → gCO2eq Converter

In [ ]:
from carbon.CarbonCalculator import CarbonCalculator

print("CarbonCalculator - Input/Output Specification")
print("="*60)

spec = {
    "component": "CarbonCalculator",
    "purpose": "Convert energy consumption to carbon emissions",
    "methods": [
        {
            "name": "estimate_from_kwh",
            "inputs": {
                "busy_kwh": "float - Energy from busy time",
                "idle_kwh": "float - Energy from idle time",
                "carbon_intensity_g_per_kwh": "float - Grid carbon intensity"
            },
            "outputs": {
                "busy_gco2eq": "float - Carbon from busy energy",
                "idle_gco2eq": "float - Carbon from idle energy",
                "total_gco2eq": "float - Total carbon emissions",
                "carbon_intensity_g_per_kwh": "float - Intensity used"
            },
            "formula": [
                "busy_gco2eq = busy_kwh * carbon_intensity_g_per_kwh",
                "idle_gco2eq = idle_kwh * carbon_intensity_g_per_kwh",
                "total_gco2eq = busy_gco2eq + idle_gco2eq"
            ],
            "use_case": "Primary method for carbon calculation from energy"
        },
        {
            "name": "estimate_carbon_footprint",
            "inputs": {
                "busy_cpu_seconds": "float",
                "idle_cpu_seconds": "float",
                "busy_power_w": "float",
                "idle_power_w": "float",
                "carbon_intensity_g_per_kwh": "float"
            },
            "outputs": {
                "total_gco2eq": "float - Total carbon emissions"
            },
            "use_case": "Direct CPU time to carbon (skips separate energy step)"
        }
    ]
}

print(json.dumps(spec, indent=2))

print("\n" + "="*60)
print("Example: Energy to Carbon Conversion")
print("="*60)

calculator = CarbonCalculator()

example_input = {
    "busy_kwh": 0.008,
    "idle_kwh": 0.000333,
    "carbon_intensity_g_per_kwh": 45.5
}

example_output = calculator.estimate_from_kwh(
    busy_kwh=example_input["busy_kwh"],
    idle_kwh=example_input["idle_kwh"],
    carbon_intensity_g_per_kwh=example_input["carbon_intensity_g_per_kwh"]
)

print("Input:")
print(json.dumps(example_input, indent=2))
print("\nOutput:")
print(json.dumps(example_output, indent=2))
print("\nCalculation:")
print(f"  Busy: {example_input['busy_kwh']} kWh × {example_input['carbon_intensity_g_per_kwh']} gCO2/kWh = {example_output['busy_gco2eq']:.3f} gCO2eq")
print(f"  Idle: {example_input['idle_kwh']} kWh × {example_input['carbon_intensity_g_per_kwh']} gCO2/kWh = {example_output['idle_gco2eq']:.3f} gCO2eq")

### 3.3 CarbonEquivalencyCalculator - gCO2eq → Relatable Comparisons

In [ ]:
from carbon.CarbonEquivalencyCalculator import CarbonEquivalencyCalculator

print("CarbonEquivalencyCalculator - Input/Output Specification")
print("="*60)

spec = {
    "component": "CarbonEquivalencyCalculator",
    "purpose": "Convert carbon emissions to relatable everyday comparisons",
    "conversion_factors": {
        "smartphone_charges": "8.22 gCO2eq per charge",
        "miles_driven": "404 gCO2eq per mile (average car)",
        "streaming_hours": "55 gCO2eq per hour (HD video)",
        "kettles_boiled": "79 gCO2eq per boil"
    },
    "methods": [
        {
            "name": "calculate_equivalencies",
            "inputs": {
                "carbon_gco2eq": "float - Total carbon emissions"
            },
            "outputs": {
                "equivalencies": {
                    "smartphone_charges": "float",
                    "miles_driven": "float",
                    "streaming_hours": "float",
                    "kettles_boiled": "float"
                },
                "total_gco2eq": "float - Input value echoed back"
            },
            "use_case": "Generate all equivalencies for UI display"
        },
        {
            "name": "get_top_equivalencies",
            "inputs": {
                "carbon_gco2eq": "float",
                "top_n": "int - How many to return (default: 3)"
            },
            "outputs": {
                "equivalencies": "List[Dict] - Top N by value, formatted"
            },
            "use_case": "Show most relevant comparisons (largest numbers)"
        },
        {
            "name": "format_equivalency",
            "inputs": {
                "equiv_type": "str - Type of equivalency",
                "value": "float - Calculated value"
            },
            "outputs": {
                "formatted": "str - Human-readable string with units"
            },
            "use_case": "Format for UI display (handles plurals, decimals)"
        }
    ]
}

print(json.dumps(spec, indent=2))

print("\n" + "="*60)
print("Example: Carbon to Equivalencies")
print("="*60)

calc = CarbonEquivalencyCalculator()

example_input = {
    "carbon_gco2eq": 379.15  # From previous example (0.37915 total)
}

example_output = calc.calculate_equivalencies(
    carbon_gco2eq=example_input["carbon_gco2eq"]
)

print("Input:")
print(json.dumps(example_input, indent=2))
print("\nOutput:")
print(json.dumps(example_output, indent=2))

print("\nFormatted for UI:")
for equiv_type, value in example_output["equivalencies"].items():
    formatted = calc.format_equivalency(equiv_type, value)
    description = calc.get_equivalency_description(equiv_type)
    print(f"  {description}: {formatted}")

## 4. Data Model Components {#models}

### 4.1 WorkspaceUsageEntry - Complete Usage Record

In [ ]:
from workspace_tracking.WorkspaceUsageEntry import WorkspaceUsageEntry

print("WorkspaceUsageEntry - Data Model Specification")
print("="*60)

spec = {
    "component": "WorkspaceUsageEntry",
    "purpose": "Structure and validate complete workspace usage record",
    "status_progression": [
        "initialized - Has workspace_id, hostname, owner only",
        "downloaded - + Has CPU data from Prometheus",
        "processed - + Has energy (kWh) and carbon (gCO2eq) data",
        "complete - + Has timestamp and user info from MongoDB"
    ],
    "fields": {
        "workspace_id": "str - MongoDB workspace identifier",
        "hostname": "str - IP address of host",
        "owner": "str - Platform username",
        "timestamp": "datetime - When this usage occurred",
        "user_info": "Dict - Complete user details",
        "busy_cpu_seconds_total": "float - Busy CPU time",
        "idle_cpu_seconds_total": "float - Idle CPU time",
        "busy_usage_kwh": "float - Energy from busy CPU",
        "idle_usage_kwh": "float - Energy from idle CPU",
        "busy_usage_gco2eq": "float - Carbon from busy energy",
        "idle_usage_gco2eq": "float - Carbon from idle energy",
        "carbon_intensity_g_per_kwh": "float - Grid intensity used",
        "carbon_equivalencies": "Dict - Relatable comparisons",
        "cpu_tdp_w": "float - CPU thermal design power",
        "status": "str - Current completion status"
    },
    "methods": [
        {
            "name": "to_dict",
            "outputs": "Dict - Complete record in JSON-serializable format",
            "use_case": "API response generation"
        },
        {
            "name": "to_json",
            "outputs": "str - JSON string of complete record",
            "use_case": "Direct JSON serialization"
        }
    ]
}

print(json.dumps(spec, indent=2))

print("\n" + "="*60)
print("Example: Complete Usage Record")
print("="*60)

# Create and populate entry
entry = WorkspaceUsageEntry(
    workspace_id="507f1f77bcf86cd799439011",
    hostname="172.16.100.50",
    owner="john.doe"
)

entry.set_timestamp(datetime(2025, 1, 15, 12, 0, 0))
entry.set_user_info({"platform_name": "john.doe", "name": "John Doe", "email": "john.doe@stfc.ac.uk"})
entry.set_cpu_seconds_total(2400.0, 1200.0)
entry.set_usage_kwh(0.008, 0.000333)
entry.set_usage_gco2eq(0.364, 0.01515, carbon_intensity=45.5)
entry.set_carbon_equivalencies({
    "smartphone_charges": 0.046,
    "miles_driven": 0.00094,
    "streaming_hours": 0.0069,
    "kettles_boiled": 0.0048
})
entry.set_cpu_tdp(65.0)

output = entry.to_dict()

print("Complete Record (to_dict output):")
print(json.dumps(output, indent=2, default=str))

print(f"\nStatus: {entry.status}")
print(f"Total Energy: {output['energy_kwh']['total']} kWh")
print(f"Total Carbon: {output['carbon_gco2eq']['total']} gCO2eq")

## 5. Widget Data Flow Examples {#widgets}

### 5.1 Stacked Bar Chart - Busy/Idle Usage Over Time

In [ ]:
print("Widget: Stacked Bar Chart")
print("="*60)

widget_spec = {
    "widget": "Stacked Bar Chart",
    "description": "Shows electricity or carbon usage over time with busy/idle breakdown",
    "user_inputs": {
        "data_type": "'electricity' | 'carbon' - What to display",
        "time_view": "'day' | 'week' | 'month' | 'year' - Granularity",
        "entity_type": "'project' | 'user' | 'group' | 'workspace' - What to track",
        "entity_id": "str - Specific entity identifier",
        "date": "Date - For day/week/month/year selection"
    },
    "api_endpoint": "/api/carbon/usage/timeseries",
    "api_query_params": {
        "entity_type": "project|user|group|workspace",
        "entity_id": "str",
        "data_type": "electricity|carbon",
        "granularity": "hour|day|week|month",
        "start_time": "ISO 8601 timestamp",
        "end_time": "ISO 8601 timestamp"
    },
    "data_source_flow": [
        "1. API receives request with filters",
        "2. MongoDB: Get all workspaces for entity in time range",
        "3. For each workspace:",
        "   a. Prometheus: Get CPU data for each time bucket",
        "   b. ElectricityEstimator: Convert CPU → kWh (if data_type='electricity')",
        "   c. CarbonIntensityAPI: Get intensity for timestamp",
        "   d. CarbonCalculator: Convert kWh → gCO2eq (if data_type='carbon')",
        "4. Aggregate by time bucket (sum busy, sum idle)",
        "5. Return time series with busy/idle arrays"
    ]
}

print(json.dumps(widget_spec, indent=2))

print("\n" + "="*60)
print("Example API Response: Day View (Hourly Bars)")
print("="*60)

# Simulated API response for user "john.doe" on 2025-01-15 (electricity mode)
api_response = {
    "entity_type": "user",
    "entity_id": "john.doe",
    "data_type": "electricity",
    "granularity": "hour",
    "start_time": "2025-01-15T00:00:00Z",
    "end_time": "2025-01-15T23:59:59Z",
    "data": {
        "labels": [f"{h}:00" for h in range(24)],
        "busy": [0.002, 0.005, 0.003, 0, 0, 0, 0, 0, 0.008, 0.012, 0.015, 0.011, 0.009, 0.013, 0.016, 0.014, 0.010, 0.007, 0.004, 0.002, 0, 0, 0, 0],
        "idle": [0.0001, 0.0002, 0.0001, 0, 0, 0, 0, 0, 0.0003, 0.0004, 0.0005, 0.0004, 0.0003, 0.0005, 0.0006, 0.0005, 0.0004, 0.0003, 0.0002, 0.0001, 0, 0, 0, 0]
    },
    "unit": "kWh",
    "total": {
        "busy": 0.131,
        "idle": 0.0048,
        "combined": 0.1358
    }
}

print("API Response:")
print(json.dumps(api_response, indent=2))

print("\nUI Chart Display:")
print("  X-axis: Time labels (0:00, 1:00, ..., 23:00)")
print("  Y-axis: Energy (kWh)")
print("  Bars: Stacked (busy on bottom, idle on top)")
print(f"  Peak hour: 14:00 with {api_response['data']['busy'][14] + api_response['data']['idle'][14]:.4f} kWh")

### 5.2 GitHub-style Heatmap - Daily Carbon for Full Year

In [ ]:
print("Widget: GitHub-style Carbon Heatmap")
print("="*60)

widget_spec = {
    "widget": "GitHub-style Heatmap",
    "description": "Shows daily carbon usage for entire year as color-coded grid",
    "user_inputs": {
        "entity_type": "'project' | 'user' | 'group' - What to track",
        "entity_id": "str - Specific entity identifier",
        "year": "int - Year to display (e.g., 2025)"
    },
    "api_endpoint": "/api/carbon/usage/heatmap",
    "api_query_params": {
        "entity_type": "project|user|group",
        "entity_id": "str",
        "year": "int"
    },
    "data_source_flow": [
        "1. API receives request for full year",
        "2. For each day of year:",
        "   a. MongoDB: Get all workspaces for entity on that day",
        "   b. For each workspace:",
        "      - Prometheus: Get CPU data",
        "      - ElectricityEstimator: CPU → kWh",
        "      - CarbonIntensityAPI: Get intensity",
        "      - CarbonCalculator: kWh → gCO2eq",
        "   c. Sum all carbon for that day",
        "3. Return dict mapping month → array of daily values",
        "4. UI handles rendering as grid with color scale"
    ],
    "color_scale": "Green (low) → Yellow → Orange → Red (high)",
    "max_redness": "Optional cap for color scale (default: auto from data)"
}

print(json.dumps(widget_spec, indent=2))

print("\n" + "="*60)
print("Example API Response: Year 2025")
print("="*60)

# Simulated API response for user "john.doe" in 2025
# Widget expects dict with month names → arrays of daily values
import random
random.seed(42)

def generate_month_data(days):
    """Generate random but realistic carbon data"""
    return [random.randint(5000, 50000) if random.random() > 0.3 else 0 for _ in range(days)]

api_response = {
    "entity_type": "user",
    "entity_id": "john.doe",
    "year": 2025,
    "unit": "gCO2eq",
    "data": {
        "January": generate_month_data(31),
        "February": generate_month_data(28),
        "March": generate_month_data(31)
        # ... remaining months truncated for brevity
    },
    "total_year_gco2eq": 3847521,
    "max_daily_gco2eq": 49876,
    "avg_daily_gco2eq": 10541
}

print("API Response Structure:")
print(json.dumps({
    "entity_type": api_response["entity_type"],
    "entity_id": api_response["entity_id"],
    "year": api_response["year"],
    "unit": api_response["unit"],
    "data": {
        "January": f"[{len(api_response['data']['January'])} values]",
        "February": f"[{len(api_response['data']['February'])} values]",
        "... (remaining months)": "..."
    },
    "total_year_gco2eq": api_response["total_year_gco2eq"],
    "max_daily_gco2eq": api_response["max_daily_gco2eq"],
    "avg_daily_gco2eq": api_response["avg_daily_gco2eq"]
}, indent=2))

print("\nSample January Values (first 7 days):")
for i, value in enumerate(api_response["data"]["January"][:7], 1):
    print(f"  Jan {i}: {value:,} gCO2eq")

print("\nUI Heatmap Display:")
print("  Grid: 7 rows (Sun-Sat) × ~53 columns (weeks)")
print("  Colors: Based on value relative to max_daily_gco2eq")
print("  Tooltip: Shows date + exact gCO2eq value on hover")

### 5.3 Workspace Tracker Card - Detailed Workspace Metrics

In [ ]:
print("Widget: Workspace Tracker Card")
print("="*60)

widget_spec = {
    "widget": "Workspace Tracker Card",
    "description": "Shows detailed metrics for a specific workspace",
    "user_inputs": {
        "workspace_id": "str - MongoDB workspace ObjectId"
    },
    "api_endpoint": "/api/carbon/workspace/{workspace_id}",
    "api_path_params": {
        "workspace_id": "str - Workspace identifier"
    },
    "data_source_flow": [
        "1. API receives workspace_id",
        "2. MongoDB: Get workspace document",
        "3. MongoDB: Get user info for owner",
        "4. Prometheus: Get latest CPU data for workspace host",
        "5. ElectricityEstimator: CPU → kWh",
        "6. CarbonIntensityAPI: Get current/latest intensity",
        "7. CarbonCalculator: kWh → gCO2eq",
        "8. CarbonEquivalencyCalculator: gCO2eq → comparisons",
        "9. Create WorkspaceUsageEntry and return to_dict()"
    ],
    "expected_output": "WorkspaceUsageEntry.to_dict() format"
}

print(json.dumps(widget_spec, indent=2))

print("\n" + "="*60)
print("Example API Response: Workspace Detail")
print("="*60)

# API returns WorkspaceUsageEntry.to_dict() output
api_response = {
    "workspace_id": "507f1f77bcf86cd799439011",
    "hostname": "172.16.100.50",
    "owner": "john.doe",
    "timestamp": "2025-01-15T12:00:00",
    "user_info": {
        "platform_name": "john.doe",
        "name": "John Doe",
        "email": "john.doe@stfc.ac.uk",
        "uid": 1001
    },
    "cpu_usage": {
        "busy_seconds": 2400.0,
        "idle_seconds": 1200.0,
        "total_seconds": 3600.0
    },
    "energy_kwh": {
        "busy": 0.008,
        "idle": 0.000333,
        "total": 0.008333
    },
    "carbon_gco2eq": {
        "busy": 0.364,
        "idle": 0.01515,
        "total": 0.37915
    },
    "carbon_intensity_g_per_kwh": 45.5,
    "carbon_equivalencies": {
        "smartphone_charges": 0.046,
        "miles_driven": 0.00094,
        "streaming_hours": 0.0069,
        "kettles_boiled": 0.0048
    },
    "cpu_tdp_w": 65.0,
    "status": "complete"
}

print("API Response:")
print(json.dumps(api_response, indent=2))

print("\nUI Card Display:")
print(f"  Header: {api_response['hostname']} (ID: {api_response['workspace_id']})")
print(f"  Status Badge: {api_response['status']}")
print(f"  Owner: {api_response['owner']}")
print(f"  Updated: {api_response['timestamp']}")
print(f"  TDP: {api_response['cpu_tdp_w']} W")
print(f"  Grid Intensity: {api_response['carbon_intensity_g_per_kwh']} g/kWh")
print("\n  Metrics Grid:")
print(f"    Energy: {api_response['energy_kwh']['total']:.6f} kWh (busy: {api_response['energy_kwh']['busy']:.6f}, idle: {api_response['energy_kwh']['idle']:.6f})")
print(f"    Carbon: {api_response['carbon_gco2eq']['total']:.3f} gCO2eq (busy: {api_response['carbon_gco2eq']['busy']:.3f}, idle: {api_response['carbon_gco2eq']['idle']:.3f})")
print(f"    CPU Time: {api_response['cpu_usage']['busy_seconds']:.0f}s busy, {api_response['cpu_usage']['idle_seconds']:.0f}s idle")
print("\n  Equivalencies:")
for key, value in api_response['carbon_equivalencies'].items():
    print(f"    {key}: {value}")

## 6. API Response Examples {#api-responses}

### 6.1 Component Integration Example

In [ ]:
print("Complete Data Flow: Single Workspace Processing")
print("="*60)

# This shows how components work together for a single workspace
# In production, this would be in ada_api endpoint handlers

workflow = {
    "step_1": {
        "component": "MongoDBClient",
        "method": "get_user_by_host_and_time",
        "input": {
            "hostname": "172.16.100.50",
            "timestamp": "2025-01-15T12:00:00Z"
        },
        "output": {
            "platform_name": "john.doe",
            "workspace_id": "507f1f77bcf86cd799439011"
        }
    },
    "step_2": {
        "component": "MongoDBClient",
        "method": "get_user_by_platform_name",
        "input": {
            "platform_name": "john.doe"
        },
        "output": {
            "platform_name": "john.doe",
            "name": "John Doe",
            "email": "john.doe@stfc.ac.uk",
            "uid": 1001
        }
    },
    "step_3": {
        "component": "PrometheusAPIClient",
        "method": "cpu_seconds_total",
        "input": {
            "timestamp": "2025-01-15T12:00:00Z",
            "cloud_project_name": "CDAaaS",
            "machine_name": "MUON",
            "host": "172.16.100.50"
        },
        "output": {
            "busy_cpu_seconds": 2400.0,
            "idle_cpu_seconds": 1200.0
        }
    },
    "step_4": {
        "component": "ElectricityEstimator",
        "method": "estimate_usage_kwh",
        "input": {
            "busy_cpu_seconds": 2400.0,
            "idle_cpu_seconds": 1200.0
        },
        "output": {
            "busy_kwh": 0.008,
            "idle_kwh": 0.000333,
            "total_kwh": 0.008333
        }
    },
    "step_5": {
        "component": "CarbonIntensityAPIClient",
        "method": "get_carbon_intensity",
        "input": {
            "timestamp": "2025-01-15T12:00:00Z"
        },
        "output": 45.5
    },
    "step_6": {
        "component": "CarbonCalculator",
        "method": "estimate_from_kwh",
        "input": {
            "busy_kwh": 0.008,
            "idle_kwh": 0.000333,
            "carbon_intensity_g_per_kwh": 45.5
        },
        "output": {
            "busy_gco2eq": 0.364,
            "idle_gco2eq": 0.01515,
            "total_gco2eq": 0.37915
        }
    },
    "step_7": {
        "component": "CarbonEquivalencyCalculator",
        "method": "calculate_equivalencies",
        "input": {
            "carbon_gco2eq": 0.37915
        },
        "output": {
            "smartphone_charges": 0.046,
            "miles_driven": 0.00094,
            "streaming_hours": 0.0069,
            "kettles_boiled": 0.0048
        }
    },
    "step_8": {
        "component": "WorkspaceUsageEntry",
        "method": "to_dict",
        "description": "Combine all data into structured record",
        "output": "Complete workspace usage record (see previous examples)"
    }
}

print("Sequential Data Flow (8 steps):")
print(json.dumps(workflow, indent=2))

print("\n" + "="*60)
print("Key Insights")
print("="*60)
print("""
1. Each component has a single, clear responsibility
2. Components are chained: output of one → input of next
3. MongoDB provides attribution (who/what)
4. Prometheus provides usage metrics (how much)
5. Carbon Intensity API provides context (grid cleanliness)
6. Calculators transform raw data → meaningful metrics
7. WorkspaceUsageEntry structures final output for API
8. For multiple workspaces, this flow runs in parallel/loop
9. For time series, aggregation happens after individual processing
10. All components are tested (113 passing unit tests)
""")

## Summary

### Component Purpose

These components are **data providers** for Ada API endpoints that serve the Ada UI widgets:

1. **Data Aggregators** fetch raw data from external sources
2. **Calculators** transform raw data into metrics (kWh, gCO2eq, comparisons)
3. **Data Models** structure and validate complete records

### Widget Support

- **Stacked Bar Chart**: Time-series aggregation of electricity/carbon with busy/idle breakdown
- **GitHub Heatmap**: Daily carbon totals for full year display
- **Workspace Card**: Complete detailed metrics for single workspace

### Integration Pattern

```
Ada UI Widget → Ada API Endpoint → Carbon Components → External APIs
                                     ↓
                              WorkspaceUsageEntry
                                     ↓
                              JSON Response
```

### Entity Aggregation

Components support querying by:
- **Project**: All workspaces in cloud project
- **User**: All workspaces owned by user
- **Group**: All workspaces owned by group members
- **Workspace**: Single workspace detail

### Time Granularities

- **Hour**: 60-minute buckets
- **Day**: 24-hour buckets
- **Week**: 7-day buckets
- **Month**: ~30-day buckets
- **Year**: Daily values for 365 days

### Next Steps

1. Integrate components into Ada API endpoints
2. Implement aggregation logic for entity types
3. Add caching for Prometheus/Carbon Intensity API calls
4. Create batch processing for time-series generation
5. Connect API endpoints to Svelte UI widgets